# Diccionarios (HashMap)

**Curso:** Estructuras de Datos (Ingeniería de Sistemas)

**Propósito:** Comprender y aplicar las ideas clave del tema, analizando complejidad y usos prácticos.

### Objetivos de aprendizaje
- Reconocer el modelo de datos y operaciones fundamentales.
- Analizar la complejidad temporal y espacial de las operaciones clave.
- Implementar y probar funciones en Python con casos de uso reales.
- Comparar ventajas y limitaciones frente a alternativas.


### Teoría esencial

Mapeo clave→valor basado en tabla hash. 
Operaciones promedio O(1) (peor caso O(n)). Claves deben ser hashables/inmutables.

### Guía de estudio
- Identifica operaciones fundamentales y su costo asintótico.
- Observa invariante(s) de la estructura (lo que siempre debe cumplirse).
- Relaciona la estructura con patrones de uso en software (colas de trabajo, caches, planificadores, etc.).

### Implementación base (desde el archivo `.py`)
El siguiente bloque contiene el código original.

In [ ]:
print('*** Sistema de Inventarios ***')

inventario = []

numero_productos = int(input('Cuantos productos deseas agregar al inventario? '))

for indice in range(numero_productos):
    print(f'Proporciona los valores del producto {indice + 1}')
    nombre = input('Nombre: ')
    precio = float(input('Precio: '))
    cantidad = int(input('Cantidad: '))
    # Creamos el diccionario con el detalle del producto
    producto = {'id': indice, 'nombre': nombre, 'precio': precio, 'cantidad': cantidad}
    # Agregamos el nuevo producto al inventario
    inventario.append(producto)

# Mostrar el inventario inicial
print(f'\nInventario inicial: {inventario}')

# Buscar un producto por id
id_buscar = int(input('\nIngresa el ID del producto a buscar: '))
producto_encontrado = None
for producto in inventario:
    if producto.get('id') == id_buscar:
        producto_encontrado = producto
        break

if producto_encontrado is not None:
    print('Información de producto encontrado: ')
    print(f'''Id: {producto_encontrado.get('id')}
    Nombre: {producto_encontrado.get('nombre')}
    Precio: {producto_encontrado.get('precio')}
    Cantidad: {producto_encontrado.get('cantidad')}''')
else:
    print(f'Producto con id {id_buscar} NO encontrado')

# Mostrar el inventario detallado
print(f'\n--- Inventario Detallado ---')
for producto in inventario:
    print(f'''Id: {producto.get('id')}
    Nombre: {producto.get('nombre')}
    Precio: ${producto.get('precio'):.2f}
    Cantidad: {producto.get('cantidad')}''')


### Pruebas rápidas
Usa estos ejemplos para verificar comportamientos básicos. Ajusta según tus funciones.

In [ ]:
# Agrega aquí tus pruebas unitarias

### Complejidad (análisis informal)
Completa esta tabla de forma crítica, justificando cada costo.

| Operación | Mejor caso | Promedio | Peor caso | Nota |
|---|---|---|---|---|
| op1 | O( ) | O( ) | O( ) |  |
| op2 | O( ) | O( ) | O( ) |  |
| op3 | O( ) | O( ) | O( ) |  |

### Ejercicios propuestos
1. Implementa pruebas unitarias con `pytest` para al menos 5 funciones/operaciones.
2. Mide empíricamente tiempos (con `timeit`) al variar n, y compara con el análisis teórico.
3. Extiende la implementación para cubrir un caso límite (overflow, colisión, degeneración, etc.).
4. Escribe una breve reflexión: ¿en qué contextos reales usarías esta estructura sobre sus alternativas?

### Desarrollo de los ejercicios
A continuación se presentan soluciones de referencia a los 4 ejercicios propuestos.

#### 1. Pruebas unitarias (pytest)
Se creó el módulo `inventory_ops.py` (operaciones sobre una lista de productos) y el archivo de pruebas `tests/test_inventory_ops.py` con más de 5 pruebas:
- Creación e ids incrementales.
- Búsqueda por id.
- Actualización de stock.
- Eliminación de producto.
- Cálculo de valor total del inventario.
- Casos de error (precio/cantidad negativa) y producto inexistente.

Ejecuta (desde la raíz del repo):
```
pytest -q Corte\ 2/ppython_sda/tests/test_inventory_ops.py
```

#### 2. Medición empírica de tiempos (timeit)
Se añadió `benchmarks/bench_inventory_ops.py` que mide `find_by_id`, `update_stock` y `remove_product` sobre el último elemento (caso que fuerza recorrer toda la lista) para tamaños n = 10, 100, 1000, 5000.
Cada operación muestra best y avg.

Ejemplo de ejecución:
```
python -m ppython_sda.benchmarks.bench_inventory_ops
```
Interpretación esperada: los tiempos deberían crecer de manera aproximadamente lineal al aumentar n (coherente con O(n) para búsqueda/actualización/eliminación en lista). `add_product` tal como está también es O(n) por usar `max` para generar id.

#### 3. Extensión: caso límite / degeneración
Se simula degeneración manteniendo la estructura como lista (sin índice hash) para observar el peor caso de búsqueda lineal. Un posible caso límite adicional sería introducir productos con datos corruptos; la función `total_value` maneja con try/except silencioso para ser tolerante a errores.

Posible mejora (no implementada a propósito para análisis): cambiar a un dict `id -> producto` y un contador incremental, lo que daría O(1) promedio en add/find/update/remove, cambiando el perfil temporal.

#### 4. Reflexión: ¿cuándo usar esta estructura?
Usar una lista de diccionarios es aceptable cuando:
- El número de elementos es pequeño (n bajo) y la simplicidad pesa más que la optimización.
- Se requiere preservar orden de inserción y recorridos frecuentes completos.
- Se hacen transformaciones secuenciales (filtrar/mapear) más que búsquedas por id repetidas.

Alternativas preferibles:
- `dict` (hash map) cuando las búsquedas/actualizaciones por clave son frecuentes (O(1) promedio) y n crece.
- `OrderedDict` (caso histórico; hoy dict ya mantiene orden) cuando necesitas operaciones de reordenamiento controlado.
- Estructuras especializadas (árbol balanceado, `bisect` sobre lista ordenada) si necesitas búsquedas por rango u ordenamientos mantenidos.
- `pandas.DataFrame` si se requieren operaciones vectorizadas, agregaciones y análisis tabular más complejo.

Resumen: la implementación actual sirve como laboratorio pedagógico para observar el costo O(n) y luego justificar migrar a un hash map real.


In [ ]:
# Benchmark rápido en la notebook (complemento al script en benchmarks)
from timeit import repeat
from ppython_sda.src.inventory_ops import add_product, find_by_id, update_stock, remove_product

resultados = []
for n in [10, 100, 1000, 3000]:
    inv = []
    for i in range(n):
        add_product(inv, f"P{i}", 1.0, 1)

    setup = "from __main__ import inv, find_by_id, update_stock, remove_product, n"  # variables ya en scope
    # Medimos buscar el último id (peor caso)
    stmt_find = f"find_by_id(inv, {n-1})"
    stmt_update = f"update_stock(inv, {n-1}, 2)"
    stmt_remove = f"remove_product(inv, {n-1})"

    for label, stmt in [("find_last", stmt_find), ("update_last", stmt_update), ("remove_last", stmt_remove)]:
        tiempos = repeat(stmt=stmt, setup=setup, repeat=3, number=30)
        mejor = min(tiempos)/30
        promedio = sum(tiempos)/len(tiempos)/30
        resultados.append((label, n, mejor, promedio))

# Mostrar resultados ordenados por n
for label, n, best_t, avg_t in resultados:
    print(f"{label:<12} n={n:<5} best={best_t:.6e}s avg={avg_t:.6e}s")

print("\nObservación: el tiempo crece aproximadamente de forma lineal con n -> consistente con O(n).")

#### Comparación empírico vs teórico
Los tiempos muestran crecimiento casi proporcional a n (cierta variabilidad por ruido del sistema). Esto valida el modelo O(n) para búsqueda/actualización/eliminación al trabajar con una lista. Una optimización natural sería cambiar a un `dict` (hash map) y medir de nuevo: la pendiente se achataría, quedando aproximadamente constante hasta que efectos de caché/memoria se hagan relevantes.

Conclusión: la implementación actual es útil para mostrar el motivo de migrar a estructuras con hashing cuando las operaciones de acceso por id son muy frecuentes.

### Referencias rápidas
- Cormen, Leiserson, Rivest, Stein. *Introduction to Algorithms* (CLRS).
- Sedgewick & Wayne. *Algorithms*.
- Documentación oficial de Python: `collections`, `heapq`, `bisect`, `array`.

*Fuente de código:* `15-Solucion-SistemaInventario-parte2-UP.py`  — Generado automáticamente el 2025-09-08 22:41.